In [1]:
!pip install torchtext==0.4.0

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
     |████████████████████████████████| 45 kB 4.9 MB/s  eta 0:00:01
  Created wheel for torchtext: filename=torchtext-0.4.0-py2-none-any.whl size=52129 sha256=761bf560878ebf302df68f2f55cec660bcf4bf5e1b67ad6ffd321c3168d324d0
  Stored in directory: /home/ec2-user/.cache/pip/wheels/86/a4/48/71e4abb43905ead16e32e61df3ca5f1da5e4d8c863d0008884
Successfully built torchtext


# [데이터 전처리]

In [2]:
max_length = 256 # sms(메시지) 최대 길이

## 1. 데이터 불러오기

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('sms.tsv', sep='\t', )
print(df.columns)
print(df.shape)

Index([u'label', u'sms'], dtype='object')
(5572, 2)


In [5]:
df.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
# 클래스 파악
classes = sorted(set(df['label']))
class_to_idx = {}

for i, c in enumerate(classes): # 모든 클래스에 대해
    class_to_idx.update({c: i})
    
nclass = len(classes)

print("# of classes: %d" %nclass)
print(classes)
print(class_to_idx)

# of classes: 2
['ham', 'spam']
{'ham': 0, 'spam': 1}


## 2. 새로운 DataFrame

### 1) 'label, sms' 만 남기기

### 2) 최대 텍스트 길이 만큼 자르기 # pandas.Series.str.slice

* '성별, 가사'만 남기려면?

In [7]:
new_df = pd.DataFrame({'label': df['label'],
                       'sms': df['sms'].str.slice( # 최대 가사 텍스트 만큼 자르기
                           start=0, stop=max_length)
                      })

### 3) 중복 제거

In [8]:
len(new_df)

5572

In [9]:
new_df = pd.DataFrame(new_df.drop_duplicates())

In [10]:
len(new_df)

5169

### 4) 셔플

In [11]:
df_shuffled = new_df.sample(frac=1).reset_index(drop=True)
df_shuffled.head()

,label,sms
0,ham,Ok. I asked for money how far
1,ham,Depends on individual lor e hair dresser say p...
2,ham,"Hmm... Dunno leh, mayb a bag 4 goigng out dat ..."
3,spam,December only! Had your mobile 11mths+? You ar...
4,ham,Spoons it is then okay?


### 5) train, test 나누기

In [13]:
# train : test = 9 : 1
# train : test = 540 : 60 -> train : valid : test = 432 : 108 : 60
train_ratio = 0.9

# train dataset
s, e = 0, int(df_shuffled.shape[0] * train_ratio) # # of rows
df_train = pd.DataFrame({'label': df_shuffled['label'][s:e],
                         'sms': df_shuffled['sms'][s:e]})
print("index for train: %d~%d" %(s,e))

# test dataset
s, e = e, e+int(df_shuffled.shape[0]*(1.0-train_ratio))
print("index for test: %d~%d" %(s,e))
df_test = pd.DataFrame({'label': df_shuffled['label'][s:e],
                        'sms': df_shuffled['sms'][s:e]})

index for train: 0~4652
index for test: 4652~5168


In [14]:
# column 수 확인
print(df_train.shape)
print(df_test.shape)

(4652, 2)
(516, 2)


### 6) 저장

In [15]:
# new_df.columns : ['label', 'sms']
df_train.to_csv('./sms.maxlen.uniq.shuf.train.tsv',
                header=False, index=False, sep='\t')
df_test.to_csv('./sms.maxlen.uniq.shuf.test.tsv',
                header=False, index=False, sep='\t')

/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/OpenSSL/crypto.py:12: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography import x509


# [데이터 로더]

In [16]:
import torch

print(torch.__version__)

1.4.0


In [17]:
import torchtext
import numpy as np

# [RNN + SMS 구현]

## 0.1. 라이브러리 임포트

In [18]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np

## 0.2. 하이퍼파라미터 셋팅

In [19]:
# Hyper-parameters
batch_size = 128
num_epochs = 10

word_vec_size = 256
dropout_p = 0.3

hidden_size = 512
num_layers = 4

## yhk 추가
learning_rate = 0.001 # 디폴트 0.001

In [20]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1. SMS train, test dataset 가져오기

In [21]:
from data_loader import DataLoader

In [23]:
loaders = DataLoader(
    train_fn = './sms.maxlen.uniq.shuf.train.tsv',
    batch_size=batch_size,
    valid_ratio=.2,
    device=-1,
    max_vocab=999999,
    min_freq=5,
)

In [24]:
test_loaders = DataLoader(
    train_fn = './sms.maxlen.uniq.shuf.test.tsv',
    batch_size=batch_size,
    valid_ratio=.01,
    device=-1,
    max_vocab=999999,
    min_freq=5,
)

## 2. 대략적인 데이터 형태

In [25]:
print("|train| =", len(loaders.train_loader.dataset),
      "|valid| =", len(loaders.valid_loader.dataset))

vocab_size = len(loaders.text.vocab)
num_classes = len(loaders.label.vocab)
print('|vocab| =', vocab_size, "|classes| =", num_classes)

('|train| =', 3722, '|valid| =', 930)
('|vocab| =', 1566, '|classes| =', 2)


## 3. 데이터 로드함수

학습시킬 때 batch_size 단위로 끊어서 로드하기 위함

### 데이터 로드함수 이해하기

In [26]:
n = 3 # 샘플로 출력할 데이터 개수
for i, data in enumerate(loaders.train_loader):
    labels = data.label
    texts = data.text
    
    if i>n:
        break
    print("[%d]" %i)
    print("한 번에 로드되는 데이터 크기:", len(labels))
    
    # 출력
    for j in range(n):
        label = labels[j].numpy() # tensor -> numpy 로 변환
        text = texts[j].numpy()
        print("label: ", label)
        print("text: ", text.shape)

[0]
('\xed\x95\x9c \xeb\xb2\x88\xec\x97\x90 \xeb\xa1\x9c\xeb\x93\x9c\xeb\x90\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xed\x81\xac\xea\xb8\xb0:', 128)
('label: ', array(0))
('text: ', (6,))
('label: ', array(0))
('text: ', (6,))
('label: ', array(0))
('text: ', (6,))
[1]
('\xed\x95\x9c \xeb\xb2\x88\xec\x97\x90 \xeb\xa1\x9c\xeb\x93\x9c\xeb\x90\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xed\x81\xac\xea\xb8\xb0:', 128)
('label: ', array(0))
('text: ', (10,))
('label: ', array(0))
('text: ', (10,))
('label: ', array(0))
('text: ', (10,))
[2]
('\xed\x95\x9c \xeb\xb2\x88\xec\x97\x90 \xeb\xa1\x9c\xeb\x93\x9c\xeb\x90\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xed\x81\xac\xea\xb8\xb0:', 128)
('label: ', array(0))
('text: ', (6,))
('label: ', array(0))
('text: ', (6,))
('label: ', array(0))
('text: ', (6,))
[3]
('\xed\x95\x9c \xeb\xb2\x88\xec\x97\x90 \xeb\xa1\x9c\xeb\x93\x9c\xeb\x90\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xed\x81\xac\xea\xb8\xb0:',

## 4. 모델 선언

In [35]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self,
                input_size, # vocab_size
                word_vec_size, # word embedding vector 차원
                hidden_size, # bidirectional LSTM의 hidden state & cell state의 size
                n_classes,
                num_layers=4, # 쌓을 레이어 개수
                dropout_p=0.3
                ):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.word_vec_size = word_vec_size
        self.hidden_size = hidden_size
        self.n_classes = n_classes
        self.num_layers = num_layers
        self.dropout_p = dropout_p
        
        # 입력 차원(vocab_size), 출력 차원(word_vec_size)
        self.emb = nn.Embedding(input_size, word_vec_size) # 부터!
        
        self.lstm = nn.LSTM(input_size=word_vec_size,
                           hidden_size=hidden_size,
                           num_layers=num_layers,
                           dropout=dropout_p, # 얼만큼 끌지. 디폴트 0
                           batch_first=True,
                           bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        # LogSoftmax + NLLoss instead of Softmax + CrossEntropy
        self.activation = nn.LogSoftmax(dim=-1) # 마지막 차원에 softmax 씌워줌
        
    def forward(self, x):
        # x: (batch_size, length)
        x = self.emb(x)
        
        # x: (batch_size, length, word_vec_size)
        x, _ = self.lstm(x) # x: output, _ : 마지막 time step의 hidden state & cell state
        
        # x: (batch_size, length, hidden_size * 2)
        # x[:,-1]: (batch_size, 1, hidden_size * 2)
        out = self.activation(self.fc(x[:, -1])) # 마지막 time step
        # self.fc(x[:, -1]): (batch_size, num_classes)
        
        return out

In [36]:
model = RNN(input_size=vocab_size,
           word_vec_size=word_vec_size,
           hidden_size=hidden_size,
           n_classes=num_classes,
           num_layers=num_layers, 
           dropout_p=dropout_p)

In [37]:
def ComputeAccr(dloader, model):
    correct = 0
    total = 0
    
    model.eval() # test mode
    for i, data in enumerate(dloader): # batch_size 만큼
        texts = data.text.to(device) # (batch_size, length)
        labels = data.label.to(device) # (batch_size, num_classes)
        
        # Forward prop.
        output = model(texts) # (batch_size, num_classes)
        _, output_index = torch.max(output, 1) # (batch_size, 1)
        
        total += labels.size(0)
        correct += (output_index == labels).sum().float()
    #print("Accuracy of Test Data: {}".format(100*correct/total))
    
    model.train()
    return (100*correct/total).numpy() # tensor -> numpy

In [38]:
print("Accuracy of Test Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of Test Data: 12.26


## 5. loss, optimizer

In [39]:
# Loss and optimizer
#loss_func = nn.CrossEntropyLoss()
loss_func = nn.NLLLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 6. 학습

In [40]:
#train the model
total_step = len(loaders.train_loader)
for epoch in range(num_epochs):
    for i, data in enumerate(loaders.train_loader):
        texts = data.text.to(device) # (batch_size, length)
        labels = data.label.to(device) # (batch_size, num_classes)
        
        print("[%d]" %i)
        
        # Forward prop.
        outputs = model(texts)
        loss = loss_func(outputs, labels)
        
        # Backward prop. & optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print('Epoch [{}/{}, Step [{}/{}]], Loss: {:.4f}, Accr: {:.2f}'
                 .format(epoch+1, num_epochs, i+1, total_step, loss.item(), ComputeAccr(loaders.valid_loader, model)))

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [1/10, Step [10/30]], Loss: 0.3267, Accr: 87.74
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [1/10, Step [20/30]], Loss: 0.2681, Accr: 87.74
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [1/10, Step [30/30]], Loss: 0.5796, Accr: 87.74
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [2/10, Step [10/30]], Loss: 1.6687, Accr: 87.74
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [2/10, Step [20/30]], Loss: 0.2947, Accr: 87.74
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [2/10, Step [30/30]], Loss: 0.1988, Accr: 88.06
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [3/10, Step [10/30]], Loss: 1.0689, Accr: 87.74
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [3/10, Step [20/30]], Loss: 0.5250, Accr: 87.96
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [3/10, Step [30/30]], Loss: 0.1276, Accr: 87.74
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [4/10, Step [10/30]], Loss: 0.2397, Accr: 89.57


## 7. 테스트

In [41]:
print("Accuracy of Valid Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of Valid Data: 96.99


## 8. 학습된 파라미터 저장

In [42]:
netname = './nets/rnn_weight.pkl'
torch.save(model, netname, )

/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


## 9. 학습된 파라미터 로드

실무에서 학습된(pretrained) 파라미터 로드하고 싶다면: 5,6,8 과정 생략한 채 실행

In [43]:
netname = './nets/rnn_weight.pkl'
model = torch.load(netname)

In [44]:
print("Accuracy of Valid Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of Valid Data: 96.99
